# Analyse Datenqualität Wärmepumpendaten

#### Imports

In [ ]:
import h5py
import pandas as pd
import numpy as np
import pickle 
from datetime import datetime
import math
import matplotlib
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import re

pd.options.mode.chained_assignment = None 

#### Globale Variablen

In [ ]:
INDEX_START = 1525270500

#### Einlesen der Daten

In [ ]:
#read in data
with open('Data/heatpump/data_heatpump.pkl', 'rb') as f:
    load_dict = pickle.load(f)

#### Erstellen eines Dataframes, welcher zur Analyse fehlender bzw. nan Zellen dient

In [ ]:
def check_nan(x):
    if x >= 0:
        return 1
    else: 
        return 0

In [ ]:
df_result = load_dict['SFH10']['index'].to_frame()
for df in load_dict:
    load_dict[df][df] = load_dict[df]['P_TOT'].apply(check_nan)
    df_result = pd.concat([df_result, load_dict[df][df]], axis=1)
df_result.set_index('index', inplace=True)
df_result

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

def plot_data_availability_histogramm(df):
    # Datenverfügbarkeit berechnen
    data_availability = df.mean(axis=0)

    # Plot
    fig, ax = plt.subplots(figsize=(12, 8))

    # Farben festlegen
    colors = ['#ff9999' if value < 1 else '#66b266' for value in data_availability]

    # Horizontales Balkendiagramm zeichnen
    data_availability.plot(kind='barh', color=colors, ax=ax)

    # Achsentitel und Plot-Titel hinzufügen
    ax.set_title('Datenverfügbarkeit')
    ax.set_ylabel('Objekte')
    ax.set_xlabel('Verfügbarkeit in %')
    ax.set_xlim(0, 1)  # x-Achse auf den Bereich 0 bis 1 setzen

    # Anzeigen
    plt.tight_layout()
    plt.show()

In [ ]:
plot_data_availability_histogramm(df_result)

#### Develop functions to analize intervalls with missing values

In [ ]:
df_10 = df_result['SFH10'].to_frame()
df_10

In [ ]:
def get_missing_intervalls(df, column):
    streak = False
    complete = False
    intervall_list = []
    for index, value in df.iterrows():
        if value.values[0] == 0:
            if streak == True:
                continue
            else:
                streak = True
                start = index
        if value.values[0] == 1:
            if streak == True:
                end = index-900
                complete = True
            

        if complete == True:
                print("entered complete")
                intervall_list.append([start, end])
                streak = False
                complete = False
    
    return intervall_list

In [ ]:
get_missing_intervalls(df_10, 'SFH10')

In [ ]:
counter = 0
for row, column in df_10.iterrows():
    if counter > 1:
        break

    #print(row)
    print(row)
    print(column.values[0])
    counter +=1 

In [ ]:
import pandas as pd

# Erstellen Sie ein Beispiel-DataFrame basierend auf Ihrem gegebenen Bild
#data = {
#    'SFH10': [0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0]
#}
#index = [1514764800, 1514765700, 1514766600, 1514767500, 1514768400, 1609454700, 1609455600, 1609456500, 1609457400, 1609458300, 1609459200]
#df = pd.DataFrame(data, index=index)

df = df_10.copy()

# Gruppen von zusammenhängenden Nullen identifizieren
df['group'] = (df['SFH10'] != df['SFH10'].shift()).cumsum()
zero_groups = df[df['SFH10'] == 0].groupby('group')

# Start- und Endindizes von zusammenhängenden Nullen ausgeben
result = []
for name, group in zero_groups:
    start_index = group.index[0]
    end_index = group.index[-1]
    result.append((start_index, end_index))

print(result)


In [ ]:
def get_missing_intervalls(df, column):
    # Gruppen von zusammenhängenden Nullen identifizieren
    df['group'] = (df[column] != df[column].shift()).cumsum()
    zero_groups = df[df[column] == 0].groupby('group')
    # Start- und Endindizes von zusammenhängenden Nullen ausgeben
    result = []
    for name, group in zero_groups:
        start_index = group.index[0]
        end_index = group.index[-1]
        result.append((start_index, end_index))

    return result

def get_missing_intervalls_length(tuple):
    length = tuple[1] - tuple[0]
    return length

#### Analysis of missing intervalls

In [ ]:
df_result

In [ ]:
#df_result.set_index('index', inplace=True)

for column in df_result.columns:
    df = df_result[column].to_frame()
    intervalls = get_missing_intervalls(df, column)
    print(column + ": " + str(intervalls))

#### reduce dataframe to common start

In [ ]:
counter = 0
for column in df_result.columns:
    df = df_result[column].to_frame()
    intervalls = get_missing_intervalls(df, column)

    if counter == 0:
        begin_index = intervalls[0][1]
        counter = 1
    else:
        if intervalls[0][1] < begin_index:
            print(column + " " + str(intervalls[0][1]))
            begin_index = intervalls[0][1]

print(begin_index)

In [ ]:
test = df_result[df_result.index >= 1525270500]

for row,value in test.iterrows():
    if test.loc[row].sum() > 30:
        print(">30: " + str(row))
        start_index = row
        break

df_final = df_result[df_result.index >= start_index]
df_final

In [ ]:
plot_data_availability_histogramm(df_final)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

def plot_data_availability(df):
    # Datenkonvertierung: Unix-Timestamp zu Datum
    df.index = pd.to_datetime(df.index, unit='s')

    # Reihenfolge der Spalten nach den Zahlen in den Objektbezeichnungen sortieren
    sorted_columns = sorted(df.columns, key=lambda x: int(x.replace("SFH", "")))

    # Prozentsätze für jede Spalte berechnen
    percentages = (df.sum() / len(df) * 100).round(2)
    percentages = percentages[sorted_columns]

    # Plot-Einstellungen, Verkleinerung der Figur
    fig, ax = plt.subplots(figsize=(7, 8))  # Kleinere Figurgröße

    # Durch jede sortierte Spalte iterieren und Datenverfügbarkeit zeichnen
    for i, column in enumerate(sorted_columns):
        # Datenverfügbarkeit
        ax.fill_between(df.index, i, i + 1, where=(df[column] == 1), color='#66D37A', step='mid')
        # Fehlende Daten
        ax.fill_between(df.index, i, i + 1, where=(df[column] == 0), color='#FF5252', step='mid')
        # Prozentsatz neben jedem Balken hinzufügen (mit zusätzlichem Leerzeichen und Abstand nach rechts)
        ax.text(df.index[-1] + pd.Timedelta(days=1), i + 0.5, f" {percentages[column]}%", verticalalignment='center', horizontalalignment='left')

    # Einstellungen
    # Anpassung der x-Limits, um den Platz für die Prozentsätze zu berücksichtigen
    ax.set_xlim([df.index.min(), df.index.max() + pd.Timedelta(days=1)])  # Reduzierter Platz für Prozentsätze
    ax.set_ylim([0, len(sorted_columns)])
    ax.set_yticks(np.arange(len(sorted_columns)) + 0.5)
    ax.set_yticklabels(sorted_columns)
    ax.set_title("Data Availability")
    ax.set_xlabel("Timestamps")
    ax.set_ylabel("Objects")

    # Farblegende hinzufügen
    ax.legend(handles=[plt.Line2D([0], [0], color='#66D37A', label='Available'),
                    plt.Line2D([0], [0], color='#FF5252', label='Missing')], loc='upper right')

    # Erweiterte Datumseinstellungen für die X-Achse mit 3-monatigem Intervall
    ax.xaxis.set_major_locator(mdates.MonthLocator(interval=3))
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))
    fig.autofmt_xdate()

    plt.tight_layout()
    plt.show()

In [ ]:
plot_data_availability(df_final)

### Closer look at data

Dev with complete data of house SFH3

In [ ]:
df_3 = load_dict["SFH3"]
df_3 = df_3[df_3['index']>INDEX_START]
df_3.set_index('index', inplace=True)
df_3.index = pd.to_datetime(df_3.index, unit='s')

Plot of all available data for whole timespan

In [ ]:
def create_consumption_plot(data, name):
    df = data.copy()
    # Erstelle eine Figur mit sekundärer Y-Achse
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Füge die Kurven für P_TOT, Q_TOT, und S_TOT zur linken Y-Achse hinzu
    fig.add_trace(
        go.Scatter(x=df['index'], y=df['P_TOT'], name='P_TOT'),
        secondary_y=False,
    )
    fig.add_trace(
        go.Scatter(x=df['index'], y=df['Q_TOT'], name='Q_TOT'),
        secondary_y=False,
    )
    fig.add_trace(
        go.Scatter(x=df['index'], y=df['S_TOT'], name='S_TOT'),
        secondary_y=False,
    )
    # Füge die Kurve für PF_TOT zur rechten Y-Achse hinzu
    fig.add_trace(
        go.Scatter(x=df['index'], y=df['PF_TOT'], name='PF_TOT', line=dict(dash='dot')),
        secondary_y=True,
    )
    # Benenne die Achsen
    fig.update_xaxes(title_text='Zeit')
    fig.update_yaxes(title_text='P_TOT, Q_TOT, S_TOT', secondary_y=False)
    fig.update_yaxes(title_text='PF_TOT', secondary_y=True)
    # Füge einen Titel hinzu und passe das Layout an
    fig.update_layout(
        title_text='Zeitliche Darstellung der Werte - {}'.format(name),
        xaxis=dict(
            tickmode='auto',
            nticks=20,
            ticks='outside',
            tickson='boundaries',
            ticklen=20
        )
    )
    # Zeige die Figur an
    fig.show()

create_consumption_plot(df_3, 'SFH3')


Resampled version

In [ ]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import pandas as pd

# Angenommen, df ist dein DataFrame.
# df = df_3.copy()

def create_resampled_consumption_plot(data, name):
    # Stelle sicher, dass 'index' in datetime umgewandelt wird und als Index gesetzt wird
    df = data.copy()
    df['index'] = pd.to_datetime(df['index'])
    df.set_index('index', inplace=True)

    # Resample der Daten auf 24-Stunden-Intervalle und berechne den Durchschnitt
    df_resampled = df.resample('24H').mean().reset_index()

    # Erstelle eine Figur mit sekundärer Y-Achse
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Füge die Kurven für P_TOT, Q_TOT, und S_TOT zur linken Y-Achse hinzu
    fig.add_trace(
        go.Scatter(x=df_resampled['index'], y=df_resampled['P_TOT'], name='P_TOT'),
        secondary_y=False,
    )

    fig.add_trace(
        go.Scatter(x=df_resampled['index'], y=df_resampled['Q_TOT'], name='Q_TOT'),
        secondary_y=False,
    )

    fig.add_trace(
        go.Scatter(x=df_resampled['index'], y=df_resampled['S_TOT'], name='S_TOT'),
        secondary_y=False,
    )

    # Füge die Kurve für PF_TOT zur rechten Y-Achse hinzu
    fig.add_trace(
        go.Scatter(x=df_resampled['index'], y=df_resampled['PF_TOT'], name='PF_TOT', line=dict(dash='dot')),
        secondary_y=True,
    )

    # Benenne die Achsen
    fig.update_xaxes(title_text='Zeit')
    fig.update_yaxes(title_text='P_TOT, Q_TOT, S_TOT', secondary_y=False)
    fig.update_yaxes(title_text='PF_TOT', secondary_y=True)

    # Füge einen Titel hinzu und passe das Layout an
    fig.update_layout(
        title_text='Zeitliche Darstellung der Werte mit 24-Stunden-Intervallen - {}'.format(name),
        xaxis=dict(
            tickmode='auto',
            nticks=20,
            ticks='outside',
            tickson='boundaries',
            ticklen=20
        )
    )

    # Zeige die Figur an
    fig.show()

In [ ]:
def sort_key(string):
    # Finde alle Zahlen im String und verbinde sie
    number = int(re.search(r'\d+', string).group())
    return number

list_strings = list(load_dict.keys())
sorted_keys = sorted(list_strings, key=sort_key)

for key in sorted_keys:
    sub_df = load_dict[key]
    sub_df = sub_df[sub_df['index']>INDEX_START]
    sub_df.set_index('index', inplace=True)
    sub_df.index = pd.to_datetime(sub_df.index, unit='s')
    sub_df.reset_index(inplace=True)
    create_resampled_consumption_plot(sub_df, key)


Plots pro Jahr

Normal

In [ ]:
def plot_year(df_year, year, name):
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    
    fig.add_trace(go.Scatter(x=df_year['index'], y=df_year['P_TOT'], name='P_TOT'), secondary_y=False)
    fig.add_trace(go.Scatter(x=df_year['index'], y=df_year['Q_TOT'], name='Q_TOT'), secondary_y=False)
    fig.add_trace(go.Scatter(x=df_year['index'], y=df_year['S_TOT'], name='S_TOT'), secondary_y=False)
    fig.add_trace(go.Scatter(x=df_year['index'], y=df_year['PF_TOT'], name='PF_TOT', line=dict(dash='dot')), secondary_y=True)
    
    # Update the layout for the subplot
    fig.update_yaxes(title_text='P_TOT, Q_TOT, S_TOT', secondary_y=False)
    fig.update_yaxes(title_text='PF_TOT', secondary_y=True)
    fig.update_xaxes(title_text='Zeit')

    # Füge einen Titel hinzu und passe das Layout an
    fig.update_layout(height=600, width=1200, title_text=f'Daten für das Jahr {year} - {name}')
    
    # Zeige die Figur an
    fig.show()

def create_plot_per_year(data, name, years=[2018,2019,2020]):
    df = data.copy()
    df['index'] = pd.to_datetime(df['index'])

    # Filtere die Daten nach Jahr
    for year in years:
        df_year = df[df['index'].dt.year == year]
        plot_year(df_year, year, name)

#create_plot_per_year(df_3, 'SFH3')

Resampled

In [ ]:
# Funktion zur Erstellung der Plots mit 24-stündiger Auflösung
def create_resampled_plot_per_year(data, year, name):
    df = data.copy()
    df['index'] = pd.to_datetime(df['index'])
    df.set_index('index', inplace=True)
    # Resample der Daten auf 24-Stunden-Intervalle und berechne den Durchschnitt
    df_resampled = df.resample('24H').mean()
    
    # Filtere nur das gewünschte Jahr
    df_year = df_resampled[df_resampled.index.year == year]

    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Füge die Daten zur Figur hinzu
    fig.add_trace(go.Scatter(x=df_year.index, y=df_year['P_TOT'], name='P_TOT'), secondary_y=False)
    fig.add_trace(go.Scatter(x=df_year.index, y=df_year['Q_TOT'], name='Q_TOT'), secondary_y=False)
    fig.add_trace(go.Scatter(x=df_year.index, y=df_year['S_TOT'], name='S_TOT'), secondary_y=False)
    fig.add_trace(go.Scatter(x=df_year.index, y=df_year['PF_TOT'], name='PF_TOT', line=dict(dash='dot')), secondary_y=True)

    # Aktualisiere die Layout-Einstellungen
    fig.update_layout(title_text=f"Durchschnittliche Werte in 24-Stunden-Intervallen für das Jahr {year} - {name}",
                      height=600, width=1400)
    fig.update_yaxes(title_text='P_TOT, Q_TOT, S_TOT', secondary_y=False)
    fig.update_yaxes(title_text='PF_TOT', secondary_y=True)

    # Zeige die Figur an
    fig.show()

# Erzeuge die Plots für jedes Jahr
create_resampled_plot_per_year(df, 2018, 'SFH3')
create_resampled_plot_per_year(df, 2019, 'SFH3')
create_resampled_plot_per_year(df, 2020, 'SFH3')